<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/excluding_weekends_and_nysa_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas_market_calendars
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal # Now this import should work
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

# ... (rest of your code)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 9.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

# Load the dataset
file_path = "/content/sample_data/SMCI.csv"
data = pd.read_csv(file_path)

# Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data.sort_values('Date', inplace=True)

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Feature Engineering: Including previous days' close prices as features
for lag in range(1, 8):
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

# Drop rows with NaN values created by the lag features
data.dropna(inplace=True)

# Define the feature set and target
X = data.drop(columns=['Close'])
y = data['Close']

# Standardize the feature set
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_cv_score = cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_cv_score = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Support Vector Machine
svm = SVR(kernel='rbf')
svm.fit(X_train, y_train)
svm_cv_score = cross_val_score(svm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')


In [7]:
# Get NYSE calendar
nyse = mcal.get_calendar('NYSE')

# Define the date range for trading days
start_date = '2024-01-01'
end_date = '2024-12-31'
trading_days = nyse.valid_days(start_date=start_date, end_date=end_date)

# Convert trading days to a list of strings for easier handling
trading_days = trading_days.strftime('%Y-%m-%d').tolist()


In [8]:
# Prepare the last data point for prediction (using the latest available data)
last_data_point = data.loc[data.index[-1]]
last_data_point_scaled = scaler.transform([last_data_point.drop('Close').values])

# Function to predict future values considering only trading days
def predict_trading_days(model, current_data, trading_days, start_date):
    predictions = []
    current_date = start_date
    trading_days_set = set(trading_days)

    while len(predictions) < 7:
        next_pred = model.predict(current_data)
        if current_date.strftime('%Y-%m-%d') in trading_days_set:
            predictions.append((current_date.strftime('%Y-%m-%d'), next_pred[0]))

        # Prepare the next data point (shift data and include new prediction)
        current_data = np.roll(current_data, -1)
        current_data[0, -1] = next_pred
        current_date += timedelta(days=1)

    return predictions

# Predict the next 7 trading days
start_date = datetime(2024, 7, 5)
predictions_lr = predict_trading_days(lr, last_data_point_scaled, trading_days, start_date)
predictions_rf = predict_trading_days(rf, last_data_point_scaled, trading_days, start_date)
predictions_svm = predict_trading_days(svm, last_data_point_scaled, trading_days, start_date)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-8-e19b43c90613>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, -1] = next_pred
<ipython-input-8-e19b43c90613>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, -1] = next_pred
<ipython-input-8-e19b43c90613>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated 

In [9]:
# Function to predict previous values considering only trading days
def predict_previous_trading_days(model, current_data, trading_days, start_date):
    predictions = []
    current_date = start_date
    trading_days_set = set(trading_days)

    while len(predictions) < 7:
        next_pred = model.predict(current_data)
        if current_date.strftime('%Y-%m-%d') in trading_days_set:
            predictions.append((current_date.strftime('%Y-%m-%d'), next_pred[0]))

        # Prepare the next data point (shift data and include new prediction)
        current_data = np.roll(current_data, 1)
        current_data[0, 0] = next_pred
        current_date -= timedelta(days=1)

    return predictions

# Predict the previous 7 trading days
start_date = datetime(2024, 7, 4)
prev_predictions_lr = predict_previous_trading_days(lr, last_data_point_scaled, trading_days, start_date)
prev_predictions_rf = predict_previous_trading_days(rf, last_data_point_scaled, trading_days, start_date)
prev_predictions_svm = predict_previous_trading_days(svm, last_data_point_scaled, trading_days, start_date)

# Reverse the previous predictions to maintain chronological order
prev_predictions_lr.reverse()
prev_predictions_rf.reverse()
prev_predictions_svm.reverse()


<ipython-input-9-f75e7b2920a9>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-9-f75e7b2920a9>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-9-f75e7b2920a9>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_data[0, 0] = next_pred
<ipython-input-9-f75e7b2920a9>:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error

In [10]:
# Combine future and previous predictions into a single DataFrame
all_predictions_lr = prev_predictions_lr + predictions_lr
all_predictions_rf = prev_predictions_rf + predictions_rf
all_predictions_svm = prev_predictions_svm + predictions_svm

# Create a DataFrame to hold all predictions
all_predictions_df = pd.DataFrame({
    'Date': [date for date, _ in all_predictions_lr],
    'Linear Regression': [pred for _, pred in all_predictions_lr],
    'Random Forest': [pred for _, pred in all_predictions_rf],
    'SVM': [pred for _, pred in all_predictions_svm]
})

print(all_predictions_df)


          Date  Linear Regression  Random Forest         SVM
0   2024-06-25       5.059352e+07    1149.988784  195.947720
1   2024-06-26       4.968286e+07    1145.268785  195.947720
2   2024-06-27       2.050783e+05    1140.568185  195.947720
3   2024-06-28       2.023275e+05    1097.998981  195.947720
4   2024-07-01       8.462415e+02     788.007297  195.947720
5   2024-07-02       8.348821e+02     794.606086  195.947720
6   2024-07-03       8.621086e+02     758.592293  195.947720
7   2024-07-05       8.465800e+02     846.504609  253.799798
8   2024-07-08       8.429873e+02     806.179391  195.947720
9   2024-07-09       8.207791e+02     786.056387  195.947720
10  2024-07-10       8.305526e+02     830.808180  195.947720
11  2024-07-11       9.054428e+02     902.039979  195.947720
12  2024-07-12       8.486813e+02     844.887593  195.947720
13  2024-07-15       9.424954e+04    1085.675675  195.947720
